In [43]:
import os
from tqdm import tqdm

os.chdir('G:/내 드라이브/projects/NLP-StockMarket/model_fin/')

## 필요모듈 import

In [44]:
import scipy as sp
import pandas as pd
import numpy as np

from konlpy.tag import Kkma    
from konlpy.tag import Hannanum    
from konlpy.tag import Okt        
from konlpy.tag import *
import nltk
import pickle

from wordcloud import WordCloud, STOPWORDS
from PIL import Image

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import os
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
from sklearn.tree import export_graphviz
from datetime import datetime, timedelta

import matplotlib.font_manager as fm
plt.rc('font', family='NanumGothic')

import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

In [45]:
kkma = Kkma()
hannanum = Hannanum()

## 자연어 데이터 import(날짜 가져오기 위해 먼저 가져옴)

In [46]:
# LG화학, 삼성SDI, SK이노베이션, 고려아연, 포스코케미칼
stock_name = '삼성SDI'

In [47]:
naver_news = pd.read_csv('./data/refined_naver_news.csv', index_col=0)
daum_news = pd.read_csv('./data/refined_daum_news.csv', index_col=0)
naver_talks = pd.read_csv(f'./data/refined_naver_talks_{stock_name}.csv', index_col=0)
daum_talks = pd.read_csv(f'./data/refined_daum_talks_{stock_name}.csv', index_col=0)
# youtube = pd.read_csv(f'./data/refined_youtube_{stock_name}.csv', index_col=0)

In [58]:
talks = pd.concat([naver_talks, daum_talks], ignore_index=True)
for i in tqdm(range(len(talks))):
    talks['Date'][i] = str(talks['Date'][i]).replace('-', '').strip()

,Date,Title
0,2021-01-01,새해 첫글이네욤
1,2021-01-01,54만원에매수해 매도중
2,2021-01-01,돈도 안들고 청약하러갔나미련한아지매네그남편누군지피곤하긋다 ㅋㅋ
3,2021-01-01,32만원에 들어간사람도 인정해줍니까
4,2021-01-01,2021 갈아온 발톱을 드러내는 한 해가 될 듯
...,...,...
37016,2022-06-16,삼성 전기차 태스크포크를 발족했다는 소식
37017,2022-06-17,긍정적인 부분은
37018,2022-06-18,삼성 반세기의 혁신을 넘어 100년 대계를 향하다
37019,2022-06-26,숏텀 롱텀 디플레이션 전쟁


In [71]:
news_df = pd.concat([naver_news, daum_news, talks
#                     ,youtube
                    ], ignore_index=True)
news_df.head()

,Date,Title
0,20210101,서학개미 사로잡은 美 ESG ETF 올해도 高高할까
1,20210101,목표가 속속 100만원대 흥 오른 삼성바이오로직스
2,20210101,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익
3,20210101,삼성전자 신고가 랠리 9만전자 가능할까
4,20210101,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다


## 주가 데이터

In [72]:
start = str(news_df.iloc[0, 0])
end = str(news_df.iloc[-1, 0])
print(start)
print(end)

20210101
20220627


### KRX에서 받은 csv로 가져오기

In [51]:
stock_df = pd.read_csv(f'./data/{stock_name}_주가_데이터.csv', index_col=0)
stock_df.head()

,일자,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,2021/01/04,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530
1,2021/01/05,686000,15000,2.24,675000,686000,658000,601692,403850137943,47172467580000,68764530
2,2021/01/06,680000,-6000,-0.87,693000,702000,671000,669150,457568981000,46759880400000,68764530
3,2021/01/07,698000,18000,2.65,688000,712000,676000,756799,526297188000,47997641940000,68764530
4,2021/01/08,739000,41000,5.87,710000,739000,705000,890113,643077420000,50816987670000,68764530


In [52]:
for i in tqdm(range(len(stock_df))):
    stock_df['일자'][i] = stock_df['일자'][i].replace('/', '')

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:05<00:00, 70.19it/s]


In [32]:
fin_stock_df = stock_df.copy()
for i in tqdm(range(len(fin_stock_df))):
    fin_stock_df.일자[i] = datetime.strptime(str(fin_stock_df.일자[i]), '%Y%m%d')   
fin_stock_df.set_index(['일자'], inplace = True)
fin_stock_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:05<00:00, 71.81it/s]


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
일자,,,,,,,,,,
2021-01-04,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530
2021-01-05,686000,15000,2.24,675000,686000,658000,601692,403850137943,47172467580000,68764530
2021-01-06,680000,-6000,-0.87,693000,702000,671000,669150,457568981000,46759880400000,68764530
2021-01-07,698000,18000,2.65,688000,712000,676000,756799,526297188000,47997641940000,68764530
2021-01-08,739000,41000,5.87,710000,739000,705000,890113,643077420000,50816987670000,68764530


In [33]:
fin_stock_df['updown'] = 0
for i in tqdm(range(len(fin_stock_df))):
    if fin_stock_df['등락률'][i] > 0:
        fin_stock_df['updown'][i] = 1
    else:
        fin_stock_df['updown'][i] = 0
fin_stock_df.head()

100%|████████████████████████████████████████████████████████████████████████████████| 369/369 [00:05<00:00, 71.33it/s]


,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,updown
일자,,,,,,,,,,,
2021-01-04,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-05,686000,15000,2.24,675000,686000,658000,601692,403850137943,47172467580000,68764530,1
2021-01-06,680000,-6000,-0.87,693000,702000,671000,669150,457568981000,46759880400000,68764530,0
2021-01-07,698000,18000,2.65,688000,712000,676000,756799,526297188000,47997641940000,68764530,1
2021-01-08,739000,41000,5.87,710000,739000,705000,890113,643077420000,50816987670000,68764530,1


## 합치기

### news_df 날짜 하루빠르게 변환

In [74]:
fin_news_df = news_df.copy()
for i in tqdm(range(len(news_df))):
    fin_news_df.Date[i] = datetime.strptime(str(news_df.Date[i]), '%Y%m%d')    
fin_news_df.head()

100%|████████████████████████████████████████████████████████████████████████| 702485/702485 [02:12<00:00, 5303.05it/s]


,Date,Title
0,2021-01-01 00:00:00,서학개미 사로잡은 美 ESG ETF 올해도 高高할까
1,2021-01-01 00:00:00,목표가 속속 100만원대 흥 오른 삼성바이오로직스
2,2021-01-01 00:00:00,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익
3,2021-01-01 00:00:00,삼성전자 신고가 랠리 9만전자 가능할까
4,2021-01-01 00:00:00,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다


In [76]:
fin_news_df['일자'] = fin_news_df['Date'] - timedelta(days=1)
fin_news_df

,Date,Title,일자
0,2021-01-01 00:00:00,서학개미 사로잡은 美 ESG ETF 올해도 高高할까,2020-12-31
1,2021-01-01 00:00:00,목표가 속속 100만원대 흥 오른 삼성바이오로직스,2020-12-31
2,2021-01-01 00:00:00,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익,2020-12-31
3,2021-01-01 00:00:00,삼성전자 신고가 랠리 9만전자 가능할까,2020-12-31
4,2021-01-01 00:00:00,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다,2020-12-31
...,...,...,...
702480,2022-06-16 00:00:00,삼성 전기차 태스크포크를 발족했다는 소식,2022-06-15
702481,2022-06-17 00:00:00,긍정적인 부분은,2022-06-16
702482,2022-06-18 00:00:00,삼성 반세기의 혁신을 넘어 100년 대계를 향하다,2022-06-17
702483,2022-06-26 00:00:00,숏텀 롱텀 디플레이션 전쟁,2022-06-25


In [77]:
fin_news_df.columns = ['Date', '제목', '일자']
fin_news_df.set_index(['일자'], inplace = True)
fin_news_df.drop(['Date'], axis=1, inplace =True)
fin_news_df

,제목
일자,
2020-12-31,서학개미 사로잡은 美 ESG ETF 올해도 高高할까
2020-12-31,목표가 속속 100만원대 흥 오른 삼성바이오로직스
2020-12-31,신생 PEF운용사 안타 떡볶이업체 SJ코레 사들여 1년만에 매각 80억 차익
2020-12-31,삼성전자 신고가 랠리 9만전자 가능할까
2020-12-31,작년 매출 신기록 쓴 카카오 엔씨 올해 더 좋다
...,...
2022-06-15,삼성 전기차 태스크포크를 발족했다는 소식
2022-06-16,긍정적인 부분은
2022-06-17,삼성 반세기의 혁신을 넘어 100년 대계를 향하다


### news_df와 stock_df 합치기

In [78]:
## 행 개수 같을때
df = fin_news_df.join(fin_stock_df, how='inner')
df.drop_duplicates('제목', inplace = True)  # 기사제목 중복 제거
print(len(df))
df

416942


,제목,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수,updown
일자,,,,,,,,,,,,
2021-01-04,고객 대접할 쿠키를 발 위에 백화점 직원들의 일탈,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,코스피 3000선 육박 배경엔 68조 고객예탁금,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,코스피 1000 돌파후 31년만에 삼천피 시대 연다,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,마켓인사이트 두산 오너家의 새옹지마 석달 전보다 50 높은값에 퓨얼셀 블록딜,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
2021-01-04,표 아시아 주요 증시 동향 1월 5일,671000,43000,6.85,660000,682000,640000,694737,460597285668,46140999630000,68764530,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-29,상반기 투자 성적 기관 외국인 동학개미 서학개미,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530,0
2022-06-29,LG엔솔 美공장 재검토 소식에 배터리 업종 동반 급락,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530,0
2022-06-29,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,570000,-11000,-1.89,571000,579000,569000,172702,98937812000,39195782100000,68764530,0


## 감성분석 모델 적용

### 감성사전 만들기

In [ ]:
import re

okt = Okt()
n_ = []
title_rename = []
for i in tqdm(range(len(df))):
    title_rename.append(re.sub("[\(\[].*?[\)\]]", "",df.iloc[i]['제목']))
    n_.append(' '.join(okt.nouns(df.iloc[i]['제목'])))
df = df.drop(['종가', '대비', '시가', '고가', '저가', '거래량', '거래대금', '시가총액', '상장주식수'], axis=1)
df['nouns'] = n_
df['제목']=title_rename
df = df[df['nouns']!='']

In [9]:
vocab = {}
cnt = 0
for i in tqdm(df['nouns']):
    i = i.split(' ')
    for j in range(len(i)):
        if i[j] in vocab or len(i[j]) <= 1:
            cnt = cnt+1
            pass
        else:
            vocab[i[j]] = 0
print(len(vocab))
print(vocab)

NameError: name 'df' is not defined

In [85]:
up = 126886
down = 136220
up_ratio = up/(up+down)
down_ratio = down/(up+down)

import collections
for i,w in tqdm(enumerate(df['nouns'])):
    w = w.split(' ')
    if (df.iloc[i]['updown']==1):
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
                continue
            vocab[noun] = vocab[noun] + down_ratio
    else:
        for j in range(len(w)):
            noun = w[j]
            if len(noun)<=1:
                continue
            vocab[noun] = vocab[noun] - up_ratio

208145it [00:22, 9326.23it/s] 


In [92]:
sent_dictionary = vocab
total = []
for i,w  in tqdm(enumerate(df['nouns'])):
    sent_score = 0
    w= w.split(' ')
    for j in w:
        if(len(j)<=1):
            continue
        elif(j not in sent_dictionary):
            continue
        else:
            sent_score = sent_score + sent_dictionary[j]
    total.append(sent_score/len(w))
df['sent_score'] = total

208145it [00:00, 253639.07it/s]


In [93]:
## 평균값 이상인 경우 1, 아닌 경우 0
df['sent_label'] = 0
df

,기사제목,등락률,updown,nouns,sent_label,sent_score
일자,,,,,,
2021-01-04,뉴욕증시 반등 성공 3대 지수 장초반 상승,6.85,1,뉴욕증시 반등 성공 지수 장 초반 상승,0,-39.775264
2021-01-04,사모펀드 입시비리 정경심 항소심 재판부 배당,6.85,1,사모펀드 입시 비리 정경 심 항소심 재판 부 배당,0,-27.438292
2021-01-04,삼천피 천스닥 시대 눈앞에 왔다,6.85,1,피 천스닥 시대 눈앞,0,2.087501
2021-01-04,개미가 끌어올린 코스피 3 000 턱밑까지 왔다,6.85,1,개미 코스피 턱밑,0,-28.674068
2021-01-04,박셀바이오 투자위험종목 지정에 6일 거래정지된다,6.85,1,박셀 바이오 투자 위험 종목 지정 거래 정지,0,-16.681373
...,...,...,...,...,...,...
2022-06-29,주식 매매 상위 종목 및 환율 2022년 6월 29일자,-1.89,0,주식 매매 상위 종목 및 환율,0,-28.242152
2022-06-29,유럽증시 경기침체 공포 커지면서 하락,-1.89,0,유럽 증시 경기 침체 공포 하락,0,-27.123780
2022-06-29,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,-1.89,0,하반기 애플 신제품 대거 출시 애플 부품 주 반등 시도,0,-15.963426


In [94]:
sum = 0
for i in tqdm(range(len(vocab))):
    sum = sum + list(vocab.values())[i]
sent_mean = sum/len(vocab)

a_ = []
for i in tqdm(range(len(df))):
    if df['sent_score'][i] > sent_mean:
        a_.append(1)
    else:
        a_.append(0)
df['sent_label'] = a_
df

100%|██████████████████████████████████████████████████████████████████████| 208145/208145 [00:00<00:00, 245985.63it/s]


,기사제목,등락률,updown,nouns,sent_label,sent_score
일자,,,,,,
2021-01-04,뉴욕증시 반등 성공 3대 지수 장초반 상승,6.85,1,뉴욕증시 반등 성공 지수 장 초반 상승,0,-39.775264
2021-01-04,사모펀드 입시비리 정경심 항소심 재판부 배당,6.85,1,사모펀드 입시 비리 정경 심 항소심 재판 부 배당,0,-27.438292
2021-01-04,삼천피 천스닥 시대 눈앞에 왔다,6.85,1,피 천스닥 시대 눈앞,1,2.087501
2021-01-04,개미가 끌어올린 코스피 3 000 턱밑까지 왔다,6.85,1,개미 코스피 턱밑,0,-28.674068
2021-01-04,박셀바이오 투자위험종목 지정에 6일 거래정지된다,6.85,1,박셀 바이오 투자 위험 종목 지정 거래 정지,0,-16.681373
...,...,...,...,...,...,...
2022-06-29,주식 매매 상위 종목 및 환율 2022년 6월 29일자,-1.89,0,주식 매매 상위 종목 및 환율,0,-28.242152
2022-06-29,유럽증시 경기침체 공포 커지면서 하락,-1.89,0,유럽 증시 경기 침체 공포 하락,0,-27.123780
2022-06-29,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,-1.89,0,하반기 애플 신제품 대거 출시 애플 부품 주 반등 시도,0,-15.963426


In [95]:
df['sent_label'].value_counts()

0    130295
1     77850
Name: sent_label, dtype: int64

In [100]:
from konlpy.tag import Mecab
# train_data, validation_data = train_test_split(df_train, test_size = 0.2, random_state = 42)

df.dropna(how='any',inplace= True)
# validation_data.dropna(how='any', inplace=True)
mecab = Mecab() 
df['tokenized'] = df['제목'].apply(mecab.morphs)
# validation_data['tokenized'] = validation_data['제목'].apply(mecab.morphs)
X_train = df['tokenized'].values
Y_train = df['sent_label'].values
# X_vali= validation_data['tokenized'].values
# Y_vali = validation_data['sent_label'].values

Exception: The MeCab dictionary does not exist at "/usr/local/lib/mecab/dic/mecab-ko-dic". Is the dictionary correctly installed?
You can also try entering the dictionary path when initializing the Mecab class: "Mecab('/some/dic/path')"

In [127]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# 빈도 수가 낮은 단어 제거
threshold = 3 # 3개 미만이면 제거

words_cnt = len(tokenizer.word_index)
rare_cnt = 0
words_freq = 0
rare_freq = 0

for key, value in tqdm(tokenizer.word_counts.items()):
    words_freq = words_freq + value

    if value < threshold:
        rare_cnt += 1
        rare_freq = rare_freq + value

print("전체 단어 수 :", words_cnt)
print("빈도가 {} 이하인 희귀 단어 수 : {}".format(threshold-1, rare_cnt))
print("희귀 단어 비율 : {}".format((rare_cnt/words_cnt) * 100))
print("희귀 단어 등장 빈도 비율 : {}".format((rare_freq / words_freq) * 100))

,re제목,sent_label
일자,,
2021-01-04,美 뉴욕증시 S P500지수 장중 최고치 기록하며 2021년 출발,0
2021-01-04,美 S P500지수 장중 최고치 찍으며 새해 출발,0
2021-01-04,새해 첫날부터 최고가 코스피 2 900 넘었다,0
2021-01-04,새해 첫 거래일 코스피 시가총액 사상 첫 2000조원 돌파,0
2021-01-04,증권사 신년사 키워드 디지털 전환 ESG 경영,0
...,...,...
2022-06-30,주식 매매 상위 종목 및 환율 2022년 6월 29일자,0
2022-06-30,유럽증시 경기침체 공포 커지면서 하락,0
2022-06-30,하반기 애플 신제품 대거 출시 애플 부품주 반등 시도,0


In [ ]:
vocab_size = words_cnt - rare_cnt + 2
print(vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV') 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
# X_vali = tokenizer.texts_to_sequences(X_vali)

In [ ]:
print('리뷰 최대 길이 :', max(len(i) for i in X_train))
# print('리뷰 평균 길이 :', sum(map(len, X_train))/ len(X_train))

In [ ]:
def below_threshold_len(max_len, nested_list):
    cnt = 0
    for s in tqdm(nested_list):
        if(len(s) <= max_len):
            cnt = cnt + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))*100))

max_len = 25
below_threshold_len(max_len, X_train)

# 패딩
X_train = pad_sequences(X_train, maxlen = max_len)
# X_vali = pad_sequences(X_vali, maxlen = max_len)

## 모델링-LSTM

In [131]:
from tensorflow.keras.layers import Embedding, Dense, GRU, LSTM, Bidirectional
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()
model.add(Embedding(vocab_size, 100))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, Y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

# model = Sequential()
# model.add(Embedding(vocab_size, 30))
# model.add(LSTM(128)) # overfitting -> dropout
# model.add(Dense(1, activation = 'sigmoid'))

# model.compile(optimizer = 'rmsprop',
#               loss = 'binary_crossentropy',
#               metrics = ['acc'])
model.summary()

  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\jinyo\\AppData\\Local\\Temp\\pip-uninstall-vn0epspn\\pip.exe'
Consider using the `--user` option or check the permissions.

ERROR: JPype1-0.5.7-cp27-none-win_amd64.whl is not a supported wheel on this platform.


In [132]:
# 저장된 모델 불러오기, 평가
loaded_model = load_model('best_model.h5')
loaded_model.evaluate(X_train, Y_train)

Exception: Install MeCab in order to use it: http://konlpy.org/en/latest/install/

In [ ]:
# 시각화

hist_dict = history.history
loss = hist_dict['loss']
val_loss = hist_dict['val_loss']
acc = hist_dict['acc']
val_acc = hist_dict['val_acc']

plt.plot(loss, 'b--', label = 'training loss')
plt.plot(val_loss, 'r--', label = 'validation loss')
plt.legend()
plt.grid()

plt.figure()
plt.plot(acc, 'b--', label = 'training accuracy')
plt.plot(val_acc, 'r--', label = 'validation accuracy')
plt.legend()
plt.grid()

plt.show()

In [ ]:
# 임의 불용어
stopwords1 = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
stopwords2 = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']
stopwords = list(set(stopwords1 + stopwords2))

def sentiment_predict(new_sentence):
    new_token = [word for word in mecab.morphs(new_sentence) if not word in stopwords]
    new_sequences = tokenizer.texts_to_sequences([new_token])
    new_pad = pad_sequences(new_sequences, maxlen = max_len)
    score = float(loaded_model.predict(new_pad))

    if score > 0.5:
        print("{} -> 긍정 ({:.2f}%)".format(new_sentence, score * 100))
    else:
        print("{} -> 부정 ({:.2f}%)".format(new_sentence, (1-score) * 100))

In [ ]:
sentiment_predict("[유럽개장] 장초반 상승세…英 1.03%↑")
sentiment_predict("[시황종합] 코스피, '침체우려'에 장중 연저점 경신…2300선 턱걸이 마감")

## vocab 확인

In [ ]:
import pandas as pd 

df = pd.DataFrame([vocab])

In [ ]:
vocab_values = list(vocab.values())
vocab_values

vocab_values2 = []

for i in tqdm(range(len(vocab_values))): 
    vocab_values2.append(round(vocab_values[i],2))
  

In [ ]:
print(max(vocab_values2))
print(min(vocab_values2))

In [ ]:
fig = plt.figure(figsize = (20,20))
plt.plot(range(len(vocab_values2)), vocab_values2)